In [1]:
# Import necessary packages
import os
import csv 
import re
import pandas as pd 
import numpy as np 
import nltk
from nltk.corpus import stopwords 

In [2]:
# Define file path for storing raw data
file_path = "/Users/maxwellcozean/Desktop/ECON 1680/Text Analysis Project/Data"

# Join paths and import csv as dataframe
df = pd.read_csv(os.path.join(file_path,"datasciencejobs.csv"),encoding='utf-8') 

In [3]:
# Drop index columns from csv file
df = df.drop(['Unnamed: 0','index'],axis=1)

# Rename type of ownership column 
df = df.rename(columns={'Type of ownership': 'Type of Ownership'})

# Consolidate null values into a single measure

# Replace -1 (int) as null
df.replace(-1, np.nan, inplace=True)

# Replace -1 (str) as null
df.replace('-1', np.nan, inplace=True) 

# Replace Unknown (str) as null
df.replace('Unknown', np.nan, inplace=True) 

# Replace Unknown / Non-Applicable (str) as null
df.replace('Unknown / Non-Applicable', np.nan, inplace=True)

# Keep company name (text prior to newline)
df['Company Name'] = df['Company Name'].str.split('\n', expand=True)[0]

In [4]:
# Split expected annual salary variable into minimum and maximum salary estimates
df[['Minimum Salary Estimate','Maximum Salary Estimate']] = df['Salary Estimate'].str.extract(r'\$(\d+)K\s*-\s*\$(\d+)K')

# Multiply mimimum salary estimate by 1000
df['Minimum Salary Estimate'] = pd.to_numeric(df['Minimum Salary Estimate'])*1000 

# Multiply maximum salary estimate by 1000
df['Maximum Salary Estimate'] = pd.to_numeric(df['Maximum Salary Estimate'])*1000

# Drop null values of minimum and maximum salary estimates (regression output)
df = df.dropna(subset=['Minimum Salary Estimate','Maximum Salary Estimate'])

# Create single measure of expected salary by taking the average of min/max salary estimates
df['Average Salary Estimate'] = (df['Minimum Salary Estimate']+df['Maximum Salary Estimate'])/2 

In [5]:
# Define skills dictionary
skills_dictionary = [
    'Analysis', 'Analysis and design', 'Analysis and reporting', 'Analysis skills',
    'Analysis support', 'Analysis techniques', 'Analysis to support', 'Analysis tools',
    'Analytic support', 'Analytical abilities', 'Analytical methods', 'Analytical projects',
    'Analytical results', 'Analytical support', 'Analytical techniques', 'Analytical tools',
    'Analyzing information', 'Quantitative analysis', 'Quantitative and qualitative',
    'anova', 'Advanced statistics', 'Data modeling',
    'Linear and logistic', 'Linear models', 'Predictive models', 'r-square',
    'Regression', 'Statistical analyses', 'Statistical analysis', 'Statistical data analysis',
    'Statistical methods', 'Statistical modeling', 'Statistical models', 'Statistical process',
    'Statistical reports', 'Statistical techniques', 'Statistical tests', 'Statistics',
    'Summarizing data', 'Algorithm', 'Algorithm design', 'Algorithm development', 'Algorithms',
    'Algorithms and applications', 'Algorithms and formulations', 'Algorithms in solving real',
    'Algorithms to match online', 'AMPL', 'Combinatorial optimization', 'Constraint based',
    'Constraint programming', 'Cplex', 'Decision analysis', 'Decision making', 'Decision problems',
    'Decision science', 'Decision sciences', 'Decision support', 'Decision support analysis',
    'Decision support applications', 'Decision support functions', 'Decision support models',
    'Decision support research', 'Decision support research analyst', 'Decision support software',
    'Decision support systems', 'Decision support tools', 'Decision tools', 'Decision trees',
    'Forecasting', 'Integer', 'Linear programming', 'Mathematical modeling', 'Mathematical models',
    'Mathematical programming', 'Nonlinear', 'Non-linear', 'Optimization', 'Quadratic',
    'Stochastic optimization', 'MATLAB', 'Model development', 'Model formulation', 'Modeling',
    'Network modeling', 'Simulate', 'Simulation', 'Simulations',
    'Data analysis', 'Data set', 'Data collection', 'Data gathering', 'Data integrity',
    'Data mining', 'Database management', 'Datamart', 'Dataset', 'ERD', 'etl', 'Large data',
    'Relational databases', 'Software applications', 'Macros', 'Microsoft Office', 'Microsoft suite', 
    'Microsoft Word', 'MS Office', 'MS Word', 'o365', 'Power Point', 'PowerPoint', 'Spreadsheet', 
    'Spreadsheets', 'Word processing','Cognos', 'fixml', 'd3', 'pentaho', 'powerbi', 'tableau',
    'Access', 'And access', 'cassandra', 'DB2', 'dbms', 'Hbase', 'Microsoft Access',
    'mongodb', 'MS Access', 'Mysql', 'Nosql', 'Oracle', 'SQL Server', 'SQL',
    'Teradata', 'tsql', 'XML', 'xsd', 'xsl', 'r', 'SAS', 'SPSS', 'STATA',
    'C', 'html', 'Linux', 'Pearl', 'Perl', 'pig', 'Python', 'ruby', 'VBA', 'Visual Basic',
    'Azure', 'Google Analytics', 'Hadoop', 'Hive', 'Salesforce', 'SAP', 'Watson'
    'Accredited college', 'BA', 'Bachelor of business administration', 'Bachelor of science',
    'Bachelors', 'Bachelors degree', 'BS', 'College or university',
    'Advanced degree', 'Doctorate', 'Graduate degree', 'Masters', 'Masters ', 'MS degree',
    'Master of business administration', 'MBA', 'MBA degree', 'Ph', 'PhD', 'PhD degree']

In [6]:
# Drop null values of job description
df = df.dropna(subset=['Job Description'])

# Define a translation table for the punctuation to be removed
table_punctuation = str.maketrans('', '', '!"#$%&\'’*+()“”–,-./:;<=>?@[\\]^_`{|}~•')

# Define set of english stopwords to be removed 
nltk_stopwords = set(stopwords.words('english'))

# Define list of stopwords to include
include = ['and', 'to', 'or']

# Define list of stopwords to remove (all but include)
stops = [word for word in nltk_stopwords if word not in include]

# Define list of link keywords 
links = ['https', 'www']

# Initialize empty list of cleaned job descriptions
clean_description = []

# Initialize empty list of extracted skills
skills_list = []

# For each job description in the dataframe 
for i, row in enumerate(df['Job Description']):
    
    # Remove punctuation using the translation table defined above
    description = row.translate(table_punctuation)
    
    # Remove carriage returns from the current description
    description = description.replace('\r',' ')
    
    # Remove newlines from the current description
    description = description.replace('\n',' ')
    
    # Remove double spaces (replace with single space)
    description = description.replace('  ',' ')
    
    # Remove stopwords 
    tokens = [word.lower() for word in nltk.tokenize.word_tokenize(description) if word.lower() not in stops]
    
    # Remove tokens with links
    tokens = [word for word in tokens if not any(link in word for link in links)]
    
    # Append tokens (with a space) to the cleaned description for each posting
    clean_description.append(' '.join(tokens))
    
    # Intitialize an empty dictionary for the counts of skills within each job description
    skill_counts = {}
    
    # For each skill in the skills dictionary
    for skill in skills_dictionary:
        
        # Update dictionary with the skills found in both the current description 
        # and the skills dictionary and the number of times they appear
        skill_counts[skill] = len(re.findall(r'\b{}\b'.format(re.escape(skill)), description, flags=re.IGNORECASE))
    
    # Initialize empty list of skills for current posting
    job_skills = []
    
    # For each skill and the number of times that skill appears in the job description
    for skill, count in skill_counts.items():
        
        # Add skills that appear multiple times by extending the job skills list
        job_skills.extend([skill] * count)
    
    # Append skills from current description to list of skills from all descriptions
    skills_list.append(job_skills)
    
    # Track progress
    print(round((i / len(df) * 100),3),'%')
    
# Create column for the cleaned job description
df['Cleaned Job Description'] = clean_description

# Create column of extracted skills for each posting
df['Skills'] = skills_list

0.0 %
0.026 %
0.051 %
0.077 %
0.103 %
0.129 %
0.154 %
0.18 %
0.206 %
0.231 %
0.257 %
0.283 %
0.309 %
0.334 %
0.36 %
0.386 %
0.412 %
0.437 %
0.463 %
0.489 %
0.514 %
0.54 %
0.566 %
0.592 %
0.617 %
0.643 %
0.669 %
0.694 %
0.72 %
0.746 %
0.772 %
0.797 %
0.823 %
0.849 %
0.874 %
0.9 %
0.926 %
0.952 %
0.977 %
1.003 %
1.029 %
1.055 %
1.08 %
1.106 %
1.132 %
1.157 %
1.183 %
1.209 %
1.235 %
1.26 %
1.286 %
1.312 %
1.337 %
1.363 %
1.389 %
1.415 %
1.44 %
1.466 %
1.492 %
1.517 %
1.543 %
1.569 %
1.595 %
1.62 %
1.646 %
1.672 %
1.698 %
1.723 %
1.749 %
1.775 %
1.8 %
1.826 %
1.852 %
1.878 %
1.903 %
1.929 %
1.955 %
1.98 %
2.006 %
2.032 %
2.058 %
2.083 %
2.109 %
2.135 %
2.16 %
2.186 %
2.212 %
2.238 %
2.263 %
2.289 %
2.315 %
2.341 %
2.366 %
2.392 %
2.418 %
2.443 %
2.469 %
2.495 %
2.521 %
2.546 %
2.572 %
2.598 %
2.623 %
2.649 %
2.675 %
2.701 %
2.726 %
2.752 %
2.778 %
2.803 %
2.829 %
2.855 %
2.881 %
2.906 %
2.932 %
2.958 %
2.984 %
3.009 %
3.035 %
3.061 %
3.086 %
3.112 %
3.138 %
3.164 %
3.189 %
3.215 %
3.241 %


24.949 %
24.974 %
25.0 %
25.026 %
25.051 %
25.077 %
25.103 %
25.129 %
25.154 %
25.18 %
25.206 %
25.231 %
25.257 %
25.283 %
25.309 %
25.334 %
25.36 %
25.386 %
25.412 %
25.437 %
25.463 %
25.489 %
25.514 %
25.54 %
25.566 %
25.592 %
25.617 %
25.643 %
25.669 %
25.694 %
25.72 %
25.746 %
25.772 %
25.797 %
25.823 %
25.849 %
25.874 %
25.9 %
25.926 %
25.952 %
25.977 %
26.003 %
26.029 %
26.055 %
26.08 %
26.106 %
26.132 %
26.157 %
26.183 %
26.209 %
26.235 %
26.26 %
26.286 %
26.312 %
26.337 %
26.363 %
26.389 %
26.415 %
26.44 %
26.466 %
26.492 %
26.517 %
26.543 %
26.569 %
26.595 %
26.62 %
26.646 %
26.672 %
26.698 %
26.723 %
26.749 %
26.775 %
26.8 %
26.826 %
26.852 %
26.878 %
26.903 %
26.929 %
26.955 %
26.98 %
27.006 %
27.032 %
27.058 %
27.083 %
27.109 %
27.135 %
27.16 %
27.186 %
27.212 %
27.238 %
27.263 %
27.289 %
27.315 %
27.341 %
27.366 %
27.392 %
27.418 %
27.443 %
27.469 %
27.495 %
27.521 %
27.546 %
27.572 %
27.598 %
27.623 %
27.649 %
27.675 %
27.701 %
27.726 %
27.752 %
27.778 %
27.803 %
27.829 %

48.971 %
48.997 %
49.023 %
49.048 %
49.074 %
49.1 %
49.126 %
49.151 %
49.177 %
49.203 %
49.228 %
49.254 %
49.28 %
49.306 %
49.331 %
49.357 %
49.383 %
49.408 %
49.434 %
49.46 %
49.486 %
49.511 %
49.537 %
49.563 %
49.588 %
49.614 %
49.64 %
49.666 %
49.691 %
49.717 %
49.743 %
49.769 %
49.794 %
49.82 %
49.846 %
49.871 %
49.897 %
49.923 %
49.949 %
49.974 %
50.0 %
50.026 %
50.051 %
50.077 %
50.103 %
50.129 %
50.154 %
50.18 %
50.206 %
50.231 %
50.257 %
50.283 %
50.309 %
50.334 %
50.36 %
50.386 %
50.412 %
50.437 %
50.463 %
50.489 %
50.514 %
50.54 %
50.566 %
50.592 %
50.617 %
50.643 %
50.669 %
50.694 %
50.72 %
50.746 %
50.772 %
50.797 %
50.823 %
50.849 %
50.874 %
50.9 %
50.926 %
50.952 %
50.977 %
51.003 %
51.029 %
51.055 %
51.08 %
51.106 %
51.132 %
51.157 %
51.183 %
51.209 %
51.235 %
51.26 %
51.286 %
51.312 %
51.337 %
51.363 %
51.389 %
51.415 %
51.44 %
51.466 %
51.492 %
51.517 %
51.543 %
51.569 %
51.595 %
51.62 %
51.646 %
51.672 %
51.698 %
51.723 %
51.749 %
51.775 %
51.8 %
51.826 %
51.852 %
51.

72.685 %
72.711 %
72.737 %
72.762 %
72.788 %
72.814 %
72.84 %
72.865 %
72.891 %
72.917 %
72.942 %
72.968 %
72.994 %
73.02 %
73.045 %
73.071 %
73.097 %
73.122 %
73.148 %
73.174 %
73.2 %
73.225 %
73.251 %
73.277 %
73.302 %
73.328 %
73.354 %
73.38 %
73.405 %
73.431 %
73.457 %
73.483 %
73.508 %
73.534 %
73.56 %
73.585 %
73.611 %
73.637 %
73.663 %
73.688 %
73.714 %
73.74 %
73.765 %
73.791 %
73.817 %
73.843 %
73.868 %
73.894 %
73.92 %
73.945 %
73.971 %
73.997 %
74.023 %
74.048 %
74.074 %
74.1 %
74.126 %
74.151 %
74.177 %
74.203 %
74.228 %
74.254 %
74.28 %
74.306 %
74.331 %
74.357 %
74.383 %
74.408 %
74.434 %
74.46 %
74.486 %
74.511 %
74.537 %
74.563 %
74.588 %
74.614 %
74.64 %
74.666 %
74.691 %
74.717 %
74.743 %
74.769 %
74.794 %
74.82 %
74.846 %
74.871 %
74.897 %
74.923 %
74.949 %
74.974 %
75.0 %
75.026 %
75.051 %
75.077 %
75.103 %
75.129 %
75.154 %
75.18 %
75.206 %
75.231 %
75.257 %
75.283 %
75.309 %
75.334 %
75.36 %
75.386 %
75.412 %
75.437 %
75.463 %
75.489 %
75.514 %
75.54 %
75.566 %
75

96.528 %
96.553 %
96.579 %
96.605 %
96.631 %
96.656 %
96.682 %
96.708 %
96.734 %
96.759 %
96.785 %
96.811 %
96.836 %
96.862 %
96.888 %
96.914 %
96.939 %
96.965 %
96.991 %
97.016 %
97.042 %
97.068 %
97.094 %
97.119 %
97.145 %
97.171 %
97.197 %
97.222 %
97.248 %
97.274 %
97.299 %
97.325 %
97.351 %
97.377 %
97.402 %
97.428 %
97.454 %
97.479 %
97.505 %
97.531 %
97.557 %
97.582 %
97.608 %
97.634 %
97.659 %
97.685 %
97.711 %
97.737 %
97.762 %
97.788 %
97.814 %
97.84 %
97.865 %
97.891 %
97.917 %
97.942 %
97.968 %
97.994 %
98.02 %
98.045 %
98.071 %
98.097 %
98.122 %
98.148 %
98.174 %
98.2 %
98.225 %
98.251 %
98.277 %
98.302 %
98.328 %
98.354 %
98.38 %
98.405 %
98.431 %
98.457 %
98.483 %
98.508 %
98.534 %
98.56 %
98.585 %
98.611 %
98.637 %
98.663 %
98.688 %
98.714 %
98.74 %
98.765 %
98.791 %
98.817 %
98.843 %
98.868 %
98.894 %
98.92 %
98.945 %
98.971 %
98.997 %
99.023 %
99.048 %
99.074 %
99.1 %
99.126 %
99.151 %
99.177 %
99.203 %
99.228 %
99.254 %
99.28 %
99.306 %
99.331 %
99.357 %
99.383 %
99.

In [7]:
# Drop columns irrelevant to analysis 
df = df.drop(['Company Name','Rating','Salary Estimate','Location',
              'Headquarters','Size','Founded','Type of Ownership','Sector',
              'Revenue','Competitors','Easy Apply'],axis=1)

In [8]:
# Save cleaned data
df.to_csv(os.path.join(file_path,'datasciencejobs_cleaned.csv'),index=False)